In [20]:
import os
import gzip
import pandas as pd
from datetime import datetime as dt
import numpy as np

"""
https://snap.stanford.edu/data/other.html

https://snap.stanford.edu/data/memetracker9.html
"""

'\nhttps://snap.stanford.edu/data/memetracker9.html\n'

In [14]:
def utc_timestamp(timestamp):
    time_format = "%Y-%m-%d %H:%M:%S"
    base_timestep = datetime.datetime(1970, 1, 1)
    timestamp = dt.strptime(timestamp, time_format)
    curr_utc = int((timestamp - base_timestep).total_seconds())
    return curr_utc

In [15]:
lines = []
with gzip.open('./raw/memetracker/quotes_2008-08.txt.gz','rt') as f:
    for line in f:
        lines.append(line)

# check P and occurance
P_oc = dict()
for line in lines:
    if line == '\n':
        continue
    [tag, content] = line.split('\n')[0].split('\t')
    if tag == 'P':
        site = content.split('http://')[1].split('/')[0]
        try:
            P_oc[site] += 1
        except KeyError:
            P_oc[site] = 1

sort_p_oc = sorted(list(P_oc.values()), reverse=True)           
print(len(sort_p_oc))
print(sort_p_oc[:10])
print(sort_p_oc[-10:])

550501
[172946, 70131, 58245, 42285, 41224, 39922, 37186, 29567, 28059, 25924]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


* The number of websites are 550501
* The largest number of website frequency can be 172946
* The smallest number of website frequency can be 1

In [54]:
print(len(np.where(numpy.array(sort_p_oc)>10000)[0]))
print(len(np.where(numpy.array(sort_p_oc)>1000)[0]))
np.where(numpy.array(sort_p_oc)>10000)[0][-1]

42
715


41

check data

In [6]:
# check P and occurance
raw_P = set()
for line in lines:
    if line == '\n':
        continue
    [tag, content] = line.split('\n')[0].split('\t')
    if tag == 'P':
        raw_P.add(content)
        
print(len(raw_P))

7340810


* The number of posts with information provided are 7340810

In [8]:
# check Q and occurance
Q_oc = dict()
for line in lines:
    if line == '\n':
        continue
    [tag, content] = line.split('\n')[0].split('\t')
    if tag == 'Q':
        try:
            Q_oc[content] += 1
        except KeyError:
            Q_oc[content] = 1

sort_q_oc = sorted(list(Q_oc.values()), reverse=True)           
print(len(sort_q_oc))
print(sort_q_oc[:10])
print(sort_q_oc[-10:])

8813559
[31287, 22948, 19328, 11149, 10543, 10413, 10152, 10152, 9279, 8948]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


* The number of phrases are 8813559
* The largest number of phrase frequency can be 31287
* The smallest number of phrase frequency can be 1

In [9]:
# check L and occurance
L_oc = dict()
for line in lines:
    if line == '\n':
        continue
    [tag, content] = line.split('\n')[0].split('\t')
    if tag == 'L':
        try:
            L_oc[content] += 1
        except KeyError:
            L_oc[content] = 1

sort_l_oc = sorted(list(L_oc.values()), reverse=True)           
print(len(sort_l_oc))
print(sort_l_oc[:10])
print(sort_l_oc[-10:])

9397791
[214062, 211698, 86214, 84251, 84222, 60849, 60212, 45442, 38116, 35406]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


* The number of linked posts are 9397791
* The largest number of linked post frequency can be 214062
* The smallest number of linked post frequency can be 1

In [10]:
raw_L = set(list(L_oc.keys()))
tem = raw_P.intersection(raw_L)
print(len(tem))

583623


* The number of posts with detailed information and being linked are 583623

we can form memes dataset and links dataset (citation network).   
   
site2userid   
userid2site 

blogurl2userid   
userid2blogurl   
   
notes in graph:   
user == site   
url == blog   
   
meme: [cascade from sourcesite to end]    
blog, time, sourcesite, meme   
   
mentionedblog: [cascade from sourceblog to end]    
blog, time, sourceblog, mentionedblog, mentionedbloguser   
   
site_frequency   
blog_frequency   

====== START processing =====   
Read data

In [32]:
# files = [file for file in os.listdir('./raw/memetracker') if '.txt.gz' in file]
files = ['./raw/memetracker/quotes_2008-08.txt.gz']
print(files)

print("starting reading raw files")
lines = []
count = 0
for file in files:
    with gzip.open(file,'rt') as f:
        for line in f:
            lines.append(line)
    count += 1
    print(count, len(files))
    if count == 1:
        break

['./raw/memetracker/quotes_2008-08.txt.gz']
starting reading raw files
1 1


* meme cascades    
get all sites frequency

In [33]:
site_frequency = dict()

for line in lines:
    if line == '\n':
        continue
    [tag, content] = line.split('\n')[0].split('\t')
    if tag == 'P':
        site = content.split('http://')[1].split('/')[0]
        try:
            site_frequency[site] += 1
        except KeyError:
            site_frequency[site] = 1

sort_site_frequency = sorted(list(site_frequency.values()), reverse=True) 
print('the number of sites:')
print(len(site_frequency))
print('top 10 sites frequency:')
print(sort_site_frequency[:10])

the number of sites:
550501
top 10 sites frequency:
[172946, 70131, 58245, 42285, 41224, 39922, 37186, 29567, 28059, 25924]


In [34]:
site2userid = dict()
userid2site = dict()

threshold = sort_site_frequency[6000 + 1]
for site in site_frequency:
    if site_frequency[site] >= threshold:
        userid2site[len(site2userid)] = site
        site2userid[site] = len(site2userid)
print(threshold)
print(len(userid2site))
print(len(site2userid))

allsites = set(list(site2userid.keys()))

110
6023
6023


In [35]:
blogs_start_line_idx = []
for i in range(len(lines)):
    line = lines[i]
    if line == '\n':
        continue
    else:
        [tag, content] = line.split('\n')[0].split('\t')
        if tag == 'P':
            site = content.split('http://')[1].split('/')[0]
            if site in allsites:
                blogs_start_line_idx.append(i)

print(len(blogs_start_line_idx))

4170881


In [51]:
blogs_start_line_idx[0]

0

In [ ]:
# check Q and occurance
Q_oc = dict()
for line in lines:
    if line == '\n':
        continue
    [tag, content] = line.split('\n')[0].split('\t')
    if tag == 'Q':
        try:
            Q_oc[content] += 1
        except KeyError:
            Q_oc[content] = 1

sort_q_oc = sorted(list(Q_oc.values()), reverse=True)           
print(len(sort_q_oc))
print(sort_q_oc[:10])
print(sort_q_oc[-10:])

In [37]:
meme_cascades = dict()
meme_cascades['meme'] = []
meme_cascades['time'] = []
meme_cascades['site'] = []
memes_frequency = dict()

for i in range(len(blogs_start_line_idx)):
    idx = blogs_start_line_idx[i]
    try:
        next_idx = blogs_start_line_idx[i+1]
    except IndexError:
        next_idx = len(lines)
    blogmemes = []
    # print(idx, next_idx)
    # print(lines[idx])
    # print(lines[next_idx])
    
    for blogidx in range(idx, next_idx-1):
        line = lines[blogidx]
        if line == '\n':
            break
        [tag, content] = line.split('\n')[0].split('\t')
        # print(line)
        if tag == 'P':
            site = content.split('http://')[1].split('/')[0]
        if tag == 'Q':
            blogmemes.append(content)
        if tag == 'T':
            blogtime = content
    # print(len(blogmemes))
    for meme in blogmemes:
        meme_cascades['meme'].append(meme)
        meme_cascades['time'].append(blogtime)
        meme_cascades['site'].append(site)
        try:
            memes_frequency[meme] += 1
        except KeyError:
            memes_frequency[meme] = 1
    # print(meme_cascades['meme'])
    # print(meme_cascades['time'])
    # print(meme_cascades['site'])

In [50]:
sort_memes_frequency = sorted(list(memes_frequency.values()), reverse=True) 
print('the number of memes:')
print(len(memes_frequency))
print('top 10 memes frequency:')
print(sort_memes_frequency[:10])
print('last 10 memes frequency:')
print(sort_memes_frequency[-10:])

the number of memes:
4534651
top 10 memes frequency:
[31287, 22848, 19328, 10543, 10413, 10152, 10152, 9279, 8948, 8948]
last 10 memes frequency:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [38]:
df = pd.DataFrame(meme_cascades)       
df.to_csv(os.path.join('./data/', "cascades_meme.csv"), index=False)
print(len(df))
df['time']=pd.to_datetime(df['time'], format="%Y-%m-%d %H:%M:%S")
df.sort_values(by=['time'], inplace=True, ascending=True)
df.to_csv(os.path.join('./data/', "cascades_meme2.csv"), index=False)

11377848


In [40]:
meme

'well maybe because you'

In [47]:
df_meme = pd.DataFrame(df.get_group('well maybe because you'))

In [48]:
df_meme.sort_values(by=['time'], inplace=True, ascending=True)

In [49]:
df_meme.drop_duplicates('site')

,meme,time,site
11377847,well maybe because you,2008-08-31 23:59:53,statueforum.com


In [62]:
list(df_meme['meme'])

['well maybe because you']